In [1]:
from __future__ import division
from pylab import *
from scipy import io
from sklearn import metrics
import os
from sys import platform

%matplotlib inline

In [2]:
#load data
SUB_ID=102
if platform=='darwin': # mac
    dir_abs='/Users/Villa/Google Drive/workspace/NAILS_analysis/P10X'
    
dir_data=dir_abs+'/p{}/'.format(SUB_ID)

epoch_all=load(dir_data+'epoch_p{}.{}'.format(SUB_ID,'npy')).item()
label_all=load(dir_data+'label_p{}.{}'.format(SUB_ID,'npy')).item()

In [3]:
# split data for training and testing
task_ID=['INSTR','WIND1','WIND2','UAV1','UAV2','BIRD']
task=task_ID[4]
epoch_train=zeros([1,epoch_all[epoch_all.keys()[0]].shape[1],epoch_all[epoch_all.keys()[0]].shape[2]])
epoch_test=zeros([1,epoch_all[epoch_all.keys()[0]].shape[1],epoch_all[epoch_all.keys()[0]].shape[2]])
label_train=zeros(1)
label_test=zeros(1)

for block in epoch_all.keys():
    if block.split('_')[1]==task: # 2,5,8 for testing
        if block.split('_')[3]=='t2' or block.split('_')[3]=='t5' or block.split('_')[3]=='t8': # for testing
            epoch_test=r_[epoch_test, epoch_all[block]]
            label_test=r_[label_test, label_all[block]]
        else: # for training
            epoch_train=r_[epoch_train, epoch_all[block]]
            label_train=r_[label_train, label_all[block]]
    else:
        continue
        
epoch_train=epoch_train[1:,:,:]
epoch_test=epoch_test[1:,:,:]
label_train=label_train[1:]
label_test=label_test[1:]  


    
# target for 1, standard for 0    
label_train[label_train==1]=0
label_train[label_train==2]=1
label_test[label_test==1]=0
label_test[label_test==2]=1

print epoch_train.shape

(1080, 32, 250)


In [13]:
from keras.models import Sequential
from keras.layers import convolutional as conv 
from keras.layers import core
from keras.optimizers import SGD, Adadelta, Adagrad
from keras import layers

In [62]:
epoch_cnn_train=epoch_train.reshape(epoch_train.shape[0],1,epoch_train.shape[1],epoch_train.shape[2])
epoch_cnn_test=epoch_test.reshape(epoch_test.shape[0],1,epoch_test.shape[1],epoch_test.shape[2])

cnn = Sequential()

cnn.add(conv.Conv2D(filters=8, kernel_size=(32,10), strides=(32,10), input_shape=(1,32,250), activation='sigmoid',
       data_format='channels_first') )   
#cnn.add(MaxPooling2D(pool_size=(5,1),strides=(2,1),border_mode='valid'))

# temporal filter 1
# cnn.add(conv.Convolution2D(10, 10,1 ,border_mode='valid',activation='sigmoid'))
# cnn.add(conv.MaxPooling2D(pool_size=(16,1),strides=(4,1),border_mode='valid'))



cnn.add(core.Flatten())
# cnn.add(core.Dense(500, init='normal',activation='sigmoid'))
# cnn.add(core.Dense(100,activation='sigmoid'))
# cnn.add( core.Dropout(0.5) )
cnn.add(core.Dense(1, activation='sigmoid'))




sgd = SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
cnn.compile(loss='binary_crossentropy', optimizer=sgd,metrics=['accuracy'])




cnn.fit(epoch_cnn_train, label_train, shuffle=False,class_weight={0:1,1:19})



Epoch 1/10
1080/1080 [==============================] - 0s - loss: 1.8600 - acc: 0.5148     
Epoch 2/10
1080/1080 [==============================] - 0s - loss: 1.4361 - acc: 0.5481     
Epoch 3/10
1080/1080 [==============================] - 0s - loss: 1.4013 - acc: 0.4926     
Epoch 4/10
1080/1080 [==============================] - 0s - loss: 1.3884 - acc: 0.4389     
Epoch 5/10
1080/1080 [==============================] - 0s - loss: 1.3709 - acc: 0.3852     
Epoch 6/10
1080/1080 [==============================] - 0s - loss: 1.3603 - acc: 0.3815     
Epoch 7/10
1080/1080 [==============================] - 0s - loss: 1.3540 - acc: 0.3815     
Epoch 8/10
1080/1080 [==============================] - 0s - loss: 1.3498 - acc: 0.3537     
Epoch 9/10
1080/1080 [==============================] - 0s - loss: 1.3468 - acc: 0.3778     
Epoch 10/10
1080/1080 [==============================] - 0s - loss: 1.3444 - acc: 0.3778     
